In [44]:
file_path = 'data_sucia_con_coordenadas.csv'
data = pd.read_csv(file_path)
data

# Drop de nan values en BANDA
data = data.dropna(subset=['BANDA'])

# Valore sunicos de BANDA
data['BANDA'].unique()

array(['E3', 'DNO', 'DO', 'E4', 'SL1'], dtype=object)

In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder

def load_and_prepare_data(file_path):
    data = pd.read_csv(file_path)

    # Filter by voluntary resignation type
    data_logistic = data[data['TIPO DE RENUNCIA'] == 'VOLUNTARIA']

    # Select features and target variable
    features_logistic = ['BANDA', 'EDAD', 'HIJOS']
    X_logistic = data_logistic[features_logistic]
    y_logistic = data_logistic['ANTIGÜEDAD']

    # Convert target variable to binary categories
    y_logistic = np.where(y_logistic < 1460, 0, 1)

    # Convert categorical variables to numerical variables
    X_logistic = pd.get_dummies(X_logistic)

    # Scale features
    scaler = StandardScaler()
    X_logistic_scaled = scaler.fit_transform(X_logistic)

    # Split data into training and test sets
    X_train_logistic, X_test_logistic, y_train_logistic, y_test_logistic = train_test_split(X_logistic_scaled, y_logistic, test_size=0.3, random_state=42)

    return X_train_logistic, X_test_logistic, y_train_logistic, y_test_logistic

def train_and_evaluate_model(X_train, X_test, y_train, y_test):
    # Create logistic regression model
    logistic_reg = LogisticRegression()
    logistic_reg.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred_logistic = logistic_reg.predict(X_test)

    # Calculate performance metrics
    accuracy = accuracy_score(y_test, y_pred_logistic, normalize=True)
    report = classification_report(y_test, y_pred_logistic)

    return accuracy, report

file_path = 'data_sucia_con_coordenadas.csv'
X_train, X_test, y_train, y_test = load_and_prepare_data(file_path)
accuracy, report = train_and_evaluate_model(X_train, X_test, y_train, y_test)

print(f"Accuracy: {accuracy}\nClassification Report: \n{report}")


Accuracy: 0.7479892761394102
Classification Report: 
              precision    recall  f1-score   support

           0       0.78      0.81      0.80       225
           1       0.70      0.65      0.67       148

    accuracy                           0.75       373
   macro avg       0.74      0.73      0.73       373
weighted avg       0.75      0.75      0.75       373



In [46]:
import gradio as gr
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np

# Your existing functions here (load_and_prepare_data and train_and_evaluate_model)

def predict_resignation(banda, edad, hijos):
    input_df = pd.DataFrame([[banda, edad, hijos]], columns=['BANDA', 'EDAD', 'HIJOS'])
    input_df = pd.get_dummies(input_df)
    for column in X_train.columns:
        if column not in input_df.columns:
            input_df[column] = 0
    input_scaled = scaler.transform(input_df)
    prediction = logistic_reg.predict(input_scaled)[0]
    return "Más probable de renunciar" if prediction == 1 else "Menos probable de renunciar"

file_path = 'data_sucia_con_coordenadas.csv'
X_train, X_test, y_train, y_test = load_and_prepare_data(file_path)
accuracy, report = train_and_evaluate_model(X_train, X_test, y_train, y_test)

logistic_reg = LogisticRegression()
logistic_reg.fit(X_train, y_train)

scaler = StandardScaler()
scaler.fit(X_train)

# BANDA options
banda_options = ['E3', 'DNO', 'DO', 'E4', 'SL1']

iface = gr.Interface(fn=predict_resignation, 
                     inputs=[gr.Dropdown(choices=banda_options, label="BANDA"), gr.Slider(label="EDAD"), gr.Slider(label="HIJOS")], 
                     outputs=gr.Textbox(label="Prediction"),
                     title="Predictor de Renuncia Voluntaria",
                     description="Introduce los detalles del empleado para predecir si es más probable que renuncie voluntariamente.")
iface.launch()


Running on local URL:  http://127.0.0.1:7879

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/pandas/core/arrays/categorical.py", line 449, in __init__
    codes, categories = factorize(values, sort=True)
  File "/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/pandas/core/algorithms.py", line 795, in factorize
    codes, uniques = factorize_array(
  File "/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/pandas/core/algorithms.py", line 595, in factorize_array
    uniques, codes = table.factorize(
  File "pandas/_libs/hashtable_class_helper.pxi", line 7280, in pandas._libs.hashtable.PyObjectHashTable.factorize
  File "pandas/_libs/hashtable_class_helper.pxi", line 7194, in pandas._libs.hashtable.PyObjectHashTable._unique
TypeError: unhashable type: 'list'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/codedspirit/anaconda3/envs/rapids-23.10